In [106]:
import pandas as pd

from utils import tag_mail

In [107]:
df = pd.read_json(r'data/gmail_azoulai_basic.txt', encoding = 'utf-8')

In [108]:
df['cat'] = df.headers.apply(tag_mail)

df.loc[df.cat=='amazon']

,snippet,date,headers,body,cat
366,"Bonjour azoulai, Votre colis contenant ELEGOO ...",2019-10-20 07:05:45,"[{'name': 'From', 'value': '""Amazon.fr"" <shipm...",Amazon.fr \n---------------------------------...,amazon
367,"Bonjour azoulai, Votre colis contenant ELEGOO ...",2019-10-20 07:03:21,"[{'name': 'From', 'value': '""Amazon.fr"" <shipm...",Amazon.fr \n---------------------------------...,amazon
368,Amazon.fr Vos commandes | Votre compte | Amazo...,2019-10-19 23:49:02,"[{'name': 'From', 'value': '""Amazon.fr"" <confi...",Confirmation d'envoi Amazon.fr \nCommande #40...,amazon
369,Amazon.fr Vos commandes | Votre compte | Amazo...,2019-10-19 08:46:41,"[{'name': 'From', 'value': '""Amazon.fr"" <confi...",Confirmation d'envoi Amazon.fr \nCommande #40...,amazon
370,Amazon.fr Vos commandes | Votre compte | Amazo...,2019-10-18 18:37:34,"[{'name': 'From', 'value': '""Amazon.fr"" <confi...",Confirmation de votre commande Amazon.fr \n \...,amazon
794,Votre abonnement de un an démarre maintenant A...,2019-05-26 13:49:54,"[{'name': 'From', 'value': 'Amazon Prime <prim...","Bonjour azoulai, \nA votre demande, nous vous...",amazon
824,Merci d&#39;avoir visité Amazon.fr ! À votre d...,2019-05-16 18:34:55,"[{'name': 'From', 'value': 'Amazon <modificati...",Merci d'avoir visité Amazon.fr ! \n \n À vot...,amazon
825,Aide sur le mot de passe Pour vérifier votre i...,2019-05-16 18:33:00,"[{'name': 'From', 'value': '""Amazon.fr"" <accou...",,amazon
862,Amazon.fr Vos commandes | Votre compte | Amazo...,2019-04-29 22:52:37,"[{'name': 'From', 'value': '""Amazon.fr"" <confi...",Confirmation d'envoi Amazon.fr \nCommande #40...,amazon
864,Amazon.fr Modification de commande Bonjour azo...,2019-04-29 17:52:04,"[{'name': 'From', 'value': '""order-update@amaz...",Mise à jour de votre commande Amazon.fr \nCom...,amazon


In [146]:
import re
df_amazon = df.loc[df.cat=='amazon']

df_amazon = df_amazon.loc[df_amazon.body.str.contains("Votre Expédition ")]

res = []

for el in df.body:
    
    tmp = {}
    
    try:
        total_cost = re.findall(r'Montant total pour cet envoi : EUR \d{0,100000},\d\d', el)[0]
#         total_cost = 
        tmp['total_cost'] = re.findall(r'\d{0,100000},\d\d',total_cost)[0]
    except:
        continue
        
    try:
        tmp['payment_tool'] = re.findall(r'Payé par (.*?): ', el)[0]
    except:
        tmp['payment_tool'] = ''
        
    products = re.findall(r'\(Vendu par (.*?) \n \n', el, re.DOTALL) #-{5,10000}
    
    if products==[]:
        continue
    
    tmp['articles'] = []
    
    for prod in products:
        res_prod = {}
        res_prod['seller'] = prod.split(') :')[0]
        
        prod = prod.split('\n ')[1:]
        
        res_prod['article'] = prod[0]
        res_prod['price'] = re.findall(r'\d{0,100000},\d\d', prod[1])[0]
        
        tmp['articles'] += [res_prod]
    
    print(el)
    print('\n\n')
    print('#######################################################')
    print('\n\n')

 Confirmation d'envoi Amazon.fr 
Commande #406-8066368-1154711 
 
 
-------------------------------------------------------------------- 
Bonjour, 
 
Ceci est le dernier colis de votre commande, les autres articles ayant été expédiés précédemment. 
 
Livraison : 
 Dimanche 20 octobre 2019 
 
 
 
Suivez votre colis en cliquant sur le lien suivant : 
										 
 
Votre commande a été expédiée à l'adresse suivante : 
 
 Paul Azoulai 
 7 Villa Emile Meyer 
 Paris, 
 75016 
 France 
 
 
Votre colis est expédié par Amazon Logistics et votre numéro de suivi est CC0011815509. 
 
Selon votre option de livraison il est possible que les informations de suivi ne soient pas encore consultables au moment où vous recevez cet e-mail. Veuillez consulter notre page d'aide pour plus d'informations sur le transporteur et obtenir l'adresse de son site de suivi en ligne : 
 
 
 
Votre Expédition 
(Vendu par ELEGOO Official - FR) : 
 ELEGOO Kit DE 37-en-1 Module Capteur V2.0 avec CD Tutorial Complet et Acce